In [1]:
import cv2                  
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import random as rn
import tensorflow as tf

#sklearn modules
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model

#dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [2]:
X=[]
Y=[]
IMG_SIZE=200
FLOWER_DAISY_DIR='flowers/daisy/'
FLOWER_SUNFLOWER_DIR='flowers/sunflower/'
FLOWER_TULIP_DIR='flowers/tulip/'
FLOWER_DANDI_DIR='flowers/dandelion/'
FLOWER_ROSE_DIR='flowers/rose/'
weights_path='./vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [3]:
def make_train_data(flower_type,DIR):
    for img in tqdm(os.listdir(DIR)):
        label = flower_type
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        X.append(np.array(img))
        Y.append(str(label))
        

In [4]:
make_train_data('Daisy',FLOWER_DAISY_DIR)
print(len(X))
make_train_data('Sunflower',FLOWER_SUNFLOWER_DIR)
print(len(X))
make_train_data('Dandelion',FLOWER_DANDI_DIR)
print(len(X))
make_train_data('Tulip',FLOWER_TULIP_DIR)
print(len(X))
make_train_data('Rose',FLOWER_ROSE_DIR)
print(len(X))

100%|██████████| 769/769 [00:02<00:00, 301.44it/s]


769


100%|██████████| 734/734 [00:02<00:00, 276.61it/s]


1503


100%|██████████| 1052/1052 [00:03<00:00, 283.00it/s]


2555


100%|██████████| 984/984 [00:03<00:00, 282.35it/s]


3539


100%|██████████| 784/784 [00:02<00:00, 311.15it/s]


4323


In [5]:
#fig,ax = plt.subplots(5,2)
#fig.set_size_inches(15,15)
#for i in range(5):
#    for j in range (2):
#        l = rn.randint(0,len(Y))
#        ax[i,j].imshow(X[l])
#        ax[i,j].set_title('Flower: '+Y[l])
#        
#plt.tight_layout()

le=LabelEncoder()
Y=le.fit_transform(Y)
Y=to_categorical(Y,5)
X = np.array(X)
X = X/255

x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
np.random.seed(42)
rn.seed(42)
tf.set_random_seed(42)

In [6]:
batch_size=16
epochs=14
base_model=VGG16(include_top=False, weights='imagenet',input_shape=(IMG_SIZE,IMG_SIZE,3), pooling='max')
#base_model.load_weights(weights_path)

model=Sequential()
model.add(base_model)
model.add(Dense(512,activation='relu'))
model.add(Dense(5,activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
datagen.fit(x_train)

In [8]:
for i in range (len(base_model.layers)):
    print (i,base_model.layers[i])

for layer in base_model.layers[11:]:
    layer.trainable=True
for layer in base_model.layers[0:11]:
    layer.trainable=False
base_model.summary()

0 <keras.engine.input_layer.InputLayer object at 0x000001C90A876A58>
1 <keras.layers.convolutional.Conv2D object at 0x000001C90A8EC048>
2 <keras.layers.convolutional.Conv2D object at 0x000001C90A8EC1D0>
3 <keras.layers.pooling.MaxPooling2D object at 0x000001C915704048>
4 <keras.layers.convolutional.Conv2D object at 0x000001C915704898>
5 <keras.layers.convolutional.Conv2D object at 0x000001C95CB28400>
6 <keras.layers.pooling.MaxPooling2D object at 0x000001C95CB44A90>
7 <keras.layers.convolutional.Conv2D object at 0x000001C95CB44470>
8 <keras.layers.convolutional.Conv2D object at 0x000001C95CB74710>
9 <keras.layers.convolutional.Conv2D object at 0x000001C95CB8E780>
10 <keras.layers.pooling.MaxPooling2D object at 0x000001C95CBC5588>
11 <keras.layers.convolutional.Conv2D object at 0x000001C95CBC5550>
12 <keras.layers.convolutional.Conv2D object at 0x000001C95CBFD240>
13 <keras.layers.convolutional.Conv2D object at 0x000001C95CC19828>
14 <keras.layers.pooling.MaxPooling2D object at 0x000001

In [ ]:
model.compile(optimizer=Adam(lr=0.00005),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()
History = model.fit_generator(datagen.flow(x_train,y_train, batch_size = batch_size),
                              epochs = epochs, validation_data = (x_test,y_test),
                             verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)
#History = model.fit(epochs = epochs, validation_data = (x_test,y_test),
#                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)
#model.fit(x_train,y_train,epochs=epochs, validation_data = (x_test,y_test), verbose = 1,
#          steps_per_epoch=x_train.shape[0] // batch_size, validation_steps = x_test.shape[0] // batch_size)
#History = model.fit(x_train, y_train, epochs = epochs, batch_size = 64, verbose=1, validation_data=(x_test, y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 2565      
Total params: 14,979,909
Trainable params: 13,244,421
Non-trainable params: 1,735,488
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/14


207/216 [===========================>..] - ETA: 1:12:46 - loss: 2.0955 - acc: 0.06 - ETA: 37:40 - loss: 1.8150 - acc: 0.2188 - ETA: 25:57 - loss: 1.7564 - acc: 0.22 - ETA: 20:06 - loss: 1.6880 - acc: 0.25 - ETA: 16:36 - loss: 1.6711 - acc: 0.30 - ETA: 14:15 - loss: 1.6802 - acc: 0.28 - ETA: 12:34 - loss: 1.6319 - acc: 0.28 - ETA: 11:17 - loss: 1.6376 - acc: 0.28 - ETA: 10:18 - loss: 1.6294 - acc: 0.27 - ETA: 9:30 - loss: 1.6010 - acc: 0.3000 - ETA: 8:51 - loss: 1.5825 - acc: 0.306 - ETA: 8:18 - loss: 1.5663 - acc: 0.328 - ETA: 7:50 - loss: 1.5416 - acc: 0.351 - ETA: 7:26 - loss: 1.5223 - acc: 0.375 - ETA: 7:05 - loss: 1.5058 - acc: 0.387 - ETA: 6:47 - loss: 1.4744 - acc: 0.410 - ETA: 6:31 - loss: 1.4652 - acc: 0.408 - ETA: 6:16 - loss: 1.4376 - acc: 0.430 - ETA: 6:03 - loss: 1.4265 - acc: 0.427 - ETA: 5:51 - loss: 1.4007 - acc: 0.440 - ETA: 5:40 - loss: 1.3905 - acc: 0.449 - ETA: 5:30 - loss: 1.3691 - acc: 0.460 - ETA: 5:21 - loss: 1.3501 - acc: 0.470 - ETA: 5:13 - loss: 1.3331 - acc: 

207/216 [===========================>..] - ETA: 8:23 - loss: 0.0754 - acc: 1.000 - ETA: 5:38 - loss: 0.3341 - acc: 0.906 - ETA: 4:42 - loss: 0.3551 - acc: 0.895 - ETA: 4:15 - loss: 0.4384 - acc: 0.859 - ETA: 3:57 - loss: 0.4560 - acc: 0.850 - ETA: 3:46 - loss: 0.4226 - acc: 0.854 - ETA: 3:37 - loss: 0.4083 - acc: 0.866 - ETA: 3:31 - loss: 0.4102 - acc: 0.859 - ETA: 3:25 - loss: 0.3958 - acc: 0.861 - ETA: 3:21 - loss: 0.3935 - acc: 0.862 - ETA: 3:19 - loss: 0.3769 - acc: 0.869 - ETA: 3:16 - loss: 0.4001 - acc: 0.864 - ETA: 3:13 - loss: 0.4117 - acc: 0.855 - ETA: 3:10 - loss: 0.4271 - acc: 0.852 - ETA: 3:07 - loss: 0.4136 - acc: 0.850 - ETA: 3:05 - loss: 0.4310 - acc: 0.835 - ETA: 3:03 - loss: 0.4195 - acc: 0.841 - ETA: 3:01 - loss: 0.4247 - acc: 0.840 - ETA: 2:59 - loss: 0.4273 - acc: 0.835 - ETA: 2:58 - loss: 0.4169 - acc: 0.840 - ETA: 2:56 - loss: 0.4118 - acc: 0.839 - ETA: 2:55 - loss: 0.3966 - acc: 0.846 - ETA: 2:53 - loss: 0.4020 - acc: 0.847 - ETA: 2:52 - loss: 0.4030 - acc: 0.846

216/216 [==============================] - ETA: 6s - loss: 0.3601 - acc: 0.867 - ETA: 5s - loss: 0.3596 - acc: 0.867 - ETA: 5s - loss: 0.3583 - acc: 0.868 - ETA: 4s - loss: 0.3586 - acc: 0.867 - ETA: 3s - loss: 0.3578 - acc: 0.868 - ETA: 2s - loss: 0.3570 - acc: 0.868 - ETA: 1s - loss: 0.3576 - acc: 0.867 - ETA: 0s - loss: 0.3578 - acc: 0.867 - 212s 984ms/step - loss: 0.3566 - acc: 0.8683 - val_loss: 0.3798 - val_acc: 0.8728
Epoch 3/14


207/216 [===========================>..] - ETA: 3:04 - loss: 0.3694 - acc: 0.750 - ETA: 2:04 - loss: 0.3151 - acc: 0.875 - ETA: 2:23 - loss: 0.3451 - acc: 0.854 - ETA: 2:32 - loss: 0.2974 - acc: 0.875 - ETA: 2:37 - loss: 0.3902 - acc: 0.862 - ETA: 2:40 - loss: 0.4408 - acc: 0.822 - ETA: 2:42 - loss: 0.3980 - acc: 0.848 - ETA: 2:43 - loss: 0.3699 - acc: 0.859 - ETA: 2:44 - loss: 0.3994 - acc: 0.854 - ETA: 2:44 - loss: 0.3897 - acc: 0.850 - ETA: 2:45 - loss: 0.3555 - acc: 0.863 - ETA: 2:45 - loss: 0.3518 - acc: 0.864 - ETA: 2:45 - loss: 0.3596 - acc: 0.860 - ETA: 2:44 - loss: 0.3652 - acc: 0.857 - ETA: 2:44 - loss: 0.3741 - acc: 0.854 - ETA: 2:44 - loss: 0.3883 - acc: 0.851 - ETA: 2:43 - loss: 0.3742 - acc: 0.856 - ETA: 2:43 - loss: 0.3602 - acc: 0.864 - ETA: 2:42 - loss: 0.3602 - acc: 0.865 - ETA: 2:42 - loss: 0.3718 - acc: 0.862 - ETA: 2:41 - loss: 0.4050 - acc: 0.854 - ETA: 2:41 - loss: 0.4029 - acc: 0.855 - ETA: 2:40 - loss: 0.3961 - acc: 0.856 - ETA: 2:39 - loss: 0.3982 - acc: 0.854

...

216/216 [==============================] - ETA: 6s - loss: 0.1149 - acc: 0.961 - ETA: 6s - loss: 0.1151 - acc: 0.961 - ETA: 5s - loss: 0.1146 - acc: 0.961 - ETA: 4s - loss: 0.1142 - acc: 0.961 - ETA: 3s - loss: 0.1137 - acc: 0.962 - ETA: 2s - loss: 0.1142 - acc: 0.961 - ETA: 1s - loss: 0.1144 - acc: 0.961 - ETA: 0s - loss: 0.1139 - acc: 0.961 - 217s 1s/step - loss: 0.1134 - acc: 0.9618 - val_loss: 0.3458 - val_acc: 0.8983
Epoch 9/14


207/216 [===========================>..] - ETA: 3:07 - loss: 0.1197 - acc: 0.937 - ETA: 3:06 - loss: 0.0726 - acc: 0.968 - ETA: 3:05 - loss: 0.1319 - acc: 0.937 - ETA: 3:04 - loss: 0.1744 - acc: 0.921 - ETA: 3:04 - loss: 0.1463 - acc: 0.937 - ETA: 3:03 - loss: 0.1403 - acc: 0.937 - ETA: 3:02 - loss: 0.1231 - acc: 0.946 - ETA: 2:46 - loss: 0.1189 - acc: 0.953 - ETA: 2:47 - loss: 0.1059 - acc: 0.958 - ETA: 2:48 - loss: 0.0959 - acc: 0.962 - ETA: 2:48 - loss: 0.1030 - acc: 0.960 - ETA: 2:48 - loss: 0.0985 - acc: 0.963 - ETA: 2:48 - loss: 0.0953 - acc: 0.966 - ETA: 2:48 - loss: 0.0900 - acc: 0.968 - ETA: 2:47 - loss: 0.0849 - acc: 0.970 - ETA: 2:47 - loss: 0.1072 - acc: 0.964 - ETA: 2:46 - loss: 0.1132 - acc: 0.959 - ETA: 2:46 - loss: 0.1106 - acc: 0.961 - ETA: 2:45 - loss: 0.1068 - acc: 0.963 - ETA: 2:45 - loss: 0.1040 - acc: 0.965 - ETA: 2:44 - loss: 0.0999 - acc: 0.967 - ETA: 2:44 - loss: 0.1074 - acc: 0.965 - ETA: 2:43 - loss: 0.1037 - acc: 0.967 - ETA: 2:43 - loss: 0.0998 - acc: 0.968

207/216 [===========================>..] - ETA: 3:10 - loss: 0.0220 - acc: 1.000 - ETA: 3:09 - loss: 0.2840 - acc: 0.937 - ETA: 3:08 - loss: 0.2197 - acc: 0.958 - ETA: 3:07 - loss: 0.2412 - acc: 0.953 - ETA: 3:06 - loss: 0.2016 - acc: 0.962 - ETA: 3:06 - loss: 0.1779 - acc: 0.958 - ETA: 3:05 - loss: 0.1615 - acc: 0.964 - ETA: 3:04 - loss: 0.1567 - acc: 0.960 - ETA: 2:50 - loss: 0.3364 - acc: 0.909 - ETA: 2:50 - loss: 0.3061 - acc: 0.918 - ETA: 2:50 - loss: 0.2865 - acc: 0.920 - ETA: 2:51 - loss: 0.2748 - acc: 0.921 - ETA: 2:50 - loss: 0.2602 - acc: 0.927 - ETA: 2:50 - loss: 0.2683 - acc: 0.924 - ETA: 2:50 - loss: 0.2869 - acc: 0.916 - ETA: 2:50 - loss: 0.2788 - acc: 0.918 - ETA: 2:49 - loss: 0.2711 - acc: 0.922 - ETA: 2:49 - loss: 0.2598 - acc: 0.927 - ETA: 2:48 - loss: 0.2508 - acc: 0.930 - ETA: 2:47 - loss: 0.2389 - acc: 0.934 - ETA: 2:47 - loss: 0.2291 - acc: 0.937 - ETA: 2:46 - loss: 0.2223 - acc: 0.937 - ETA: 2:46 - loss: 0.2136 - acc: 0.940 - ETA: 2:45 - loss: 0.2213 - acc: 0.937

216/216 [==============================] - ETA: 7s - loss: 0.1123 - acc: 0.963 - ETA: 6s - loss: 0.1122 - acc: 0.963 - ETA: 5s - loss: 0.1121 - acc: 0.963 - ETA: 4s - loss: 0.1131 - acc: 0.962 - ETA: 3s - loss: 0.1147 - acc: 0.962 - ETA: 2s - loss: 0.1146 - acc: 0.962 - ETA: 1s - loss: 0.1145 - acc: 0.962 - ETA: 0s - loss: 0.1147 - acc: 0.961 - 220s 1s/step - loss: 0.1150 - acc: 0.9615 - val_loss: 0.3441 - val_acc: 0.8983
Epoch 11/14


207/216 [===========================>..] - ETA: 3:09 - loss: 0.1138 - acc: 0.937 - ETA: 3:09 - loss: 0.1885 - acc: 0.906 - ETA: 3:08 - loss: 0.1503 - acc: 0.937 - ETA: 3:07 - loss: 0.1130 - acc: 0.953 - ETA: 3:06 - loss: 0.0912 - acc: 0.962 - ETA: 3:06 - loss: 0.0791 - acc: 0.968 - ETA: 3:05 - loss: 0.0961 - acc: 0.955 - ETA: 3:04 - loss: 0.1019 - acc: 0.953 - ETA: 3:03 - loss: 0.1152 - acc: 0.944 - ETA: 2:50 - loss: 0.1039 - acc: 0.950 - ETA: 2:51 - loss: 0.0947 - acc: 0.954 - ETA: 2:51 - loss: 0.1081 - acc: 0.953 - ETA: 2:51 - loss: 0.1055 - acc: 0.951 - ETA: 2:51 - loss: 0.1053 - acc: 0.950 - ETA: 2:50 - loss: 0.1000 - acc: 0.954 - ETA: 2:50 - loss: 0.0956 - acc: 0.957 - ETA: 2:49 - loss: 0.0903 - acc: 0.959 - ETA: 2:49 - loss: 0.0859 - acc: 0.961 - ETA: 2:48 - loss: 0.0845 - acc: 0.963 - ETA: 2:48 - loss: 0.0855 - acc: 0.962 - ETA: 2:47 - loss: 0.0826 - acc: 0.964 - ETA: 2:46 - loss: 0.0899 - acc: 0.963 - ETA: 2:46 - loss: 0.0864 - acc: 0.964 - ETA: 2:45 - loss: 0.0830 - acc: 0.966

In [ ]:
model.save('flowers_BN_0530_Test.h5')

plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

plt.plot(History.history['acc'])
plt.plot(History.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
# getting predictions on val set.
import warnings
#model = load_model('./flowers_BN.h5')
pred = model.predict(x_test)
pred_digits = np.argmax(pred,axis=1)

# now storing some properly as well as misclassified indexes'.
i=0
prop_class=[]
mis_class=[]

for i in range(len(y_test)):
    if(np.argmax(y_test[i])==pred_digits[i]):
        prop_class.append(i)
    if(len(prop_class)==8):
        break

i=0
for i in range(len(y_test)):
    if(not np.argmax(y_test[i])==pred_digits[i]):
        mis_class.append(i)
    if(len(mis_class)==8):
        break

In [ ]:
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

count=0
fig,ax=plt.subplots(4,2)
fig.set_size_inches(15,15)
for i in range (4):
    for j in range (2):
        ax[i,j].imshow(x_test[prop_class[count]])
        ax[i,j].set_title("Predicted Flower : "+str(le.inverse_transform([pred_digits[prop_class[count]]]))+"\n"+"Actual Flower : "+str(le.inverse_transform([np.argmax([y_test[prop_class[count]]])])))
        plt.tight_layout()
        count+=1

In [ ]:
count=0
fig,ax=plt.subplots(4,2)
fig.set_size_inches(15,15)
for i in range (4):
    for j in range (2):
        ax[i,j].imshow(x_test[mis_class[count]])
        ax[i,j].set_title("Predicted Flower : "+str(le.inverse_transform([pred_digits[mis_class[count]]]))+"\n"+"Actual Flower : "+str(le.inverse_transform([np.argmax([y_test[mis_class[count]]])])))
        plt.tight_layout()
        count+=1